# Video #10

In [1]:
import numpy as np
import pandas as pd 

In [3]:
import cvxpy as cvx

In [10]:
portfolio = pd.Series([0.035, 0.017, 0.029, 0.093, 0.033], index=['A', 'B', 'C', 'D', 'E'])
portfolio

A    0.035
B    0.017
C    0.029
D    0.093
E    0.033
dtype: float64

In [11]:
covariance =  pd.DataFrame(
    {
        'A':[0.0014, 0.0008, 0.0005, 0.0007, 0.0009],
        'B':[0.0007, 0.0008, 0.0005, 0.0007, 0.0009],
        'C':[0.0004, 0.0008, 0.0005, 0.0007, 0.0009],
        'D':[0.0007, 0.0008, 0.0005, 0.0007, 0.0009],
        'E':[0.0009, 0.0008, 0.0005, 0.0007, 0.0009],
    }
)
covariance['Q'] = [0.0014, 0.0008, 0.0005, 0.0007, 0.0009]

covariance

,A,B,C,D,E,Q
0,0.0014,0.0007,0.0004,0.0007,0.0009,0.0014
1,0.0008,0.0008,0.0008,0.0008,0.0008,0.0008
2,0.0005,0.0005,0.0005,0.0005,0.0005,0.0005
3,0.0007,0.0007,0.0007,0.0007,0.0007,0.0007
4,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009


# Sharpe Ratio 
### L19


In [12]:
f = open("/Users/manwaiwong/key/quandl.txt","r")

if f.mode == 'r':
    c = f.read() 
    #print(c)


In [13]:
# from pandas.io import data, wb # becomes
import os
import pandas_datareader.data as pdr
import datetime as dt
import requests_cache

# print(pdr.get_data_fred('GS10'))

# option 
tickers = ["HKEX/01810","HKEX/09988","HKEX/00700","HKEX/02342", "HKEX/00552"]
# ticker = "HKEX/HSINX2019"
# ticker = "HKEX/00001"
# 
# ticker = "0001.hk"

expire_after = dt.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
start = dt.date.today() - dt.timedelta(5)
end = dt.date.today() 
print(end)
qkey = c


close_prices = pd.DataFrame() # dataframe to store close price of each ticker
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
## loop tickers get data
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            #print(i)
            temp = pdr.DataReader(tickers[i], 'quandl', start, end, api_key=qkey, session=session)
            #print(temp)
            close_prices[tickers[i]] = temp["PreviousClose"]
            temp.dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1

close_prices['TotalPosition'] = close_prices.sum(axis = 1)
close_prices['DailyReturn'] = close_prices['TotalPosition'].pct_change(1)
sharpe_ratio = close_prices['DailyReturn'].mean() / close_prices['DailyReturn'].std()
sharpe_ratio

annual_sharpe_ratio = (252**0.5) * sharpe_ratio
annual_sharpe_ratio


2020-02-18


-3.8577666036073013

# Optimization Portfolio
### L20 Two-Asset Portfolio Optimization
#### Derivation of Optimal Weights on a Two-Asset Portfolio
x_1 + x_2 + x_3 + ... + x_n = 1
where x is the weight of various assets in th portfolio
 
### (1) σ<sup>2</sup><sub>P</sub> =  x<sup>2</sup><sub>A</sub>σ<sup>2</sup><sub>A</sub> + x<sup>2</sup><sub>B</sub>σ<sup>2</sup><sub>B</sub> + 2x<sub>A</sub>x<sub>B</sub>σ<sub>A</sub>σ<sub>B</sub>ρ<sub>r<sub>A</sub>r<sub>B</sub></sub>

σ<sup>2</sup><sub>P</sub> = is the portfolio variance \
σ<sup>2</sup><sub>A</sub> = is the variance of the return of Stock A \
σ<sup>2</sup><sub>B</sub> = is the variance of the return of Stock B \
ρ<sub>r<sub>A</sub>r<sub>B</sub></sub> = is the correlation between  the returns of stocks A and B. \
\
\
\
constraints is : 
###  (2) x<sub>A</sub> + x<sub>B</sub> = 1

put (2) in (1) \ 
then :
### x<sub>B</sub> = 1 - x<sub>A</sub> 



## μ<sub>P</sub> = μ<sub>A</sub>x<sub>A</sub> + μ<sub>B</sub>x<sub>B</sub> 

## μ is portfolio mean

# CVXPY

## Obsolete method

In [16]:
# quandl 
tickers = ["HKEX/01810","HKEX/02342", "HKEX/00552"]
expire_after = dt.timedelta(days=3)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
start = dt.date.today() - dt.timedelta(5)
end = dt.date.today() 
qkey = c


close_prices = pd.DataFrame() # dataframe to store close price of each ticker
share_volumes = pd.DataFrame()
attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
## loop tickers get data
while len(tickers) != 0 and attempt <= 5:
    tickers = [j for j in tickers if j not in drop] # removing stocks whose data has been extracted from the ticker list
    for i in range(len(tickers)):
        try:
            temp = pdr.DataReader(tickers[i], 'quandl', start, end, api_key=qkey, session=session)
            print(temp)
            #print(temp)
            close_prices[tickers[i]] = temp["PreviousClose"]
            share_volumes[tickers[i]] = temp["ShareVolume000"]
            temp.dropna(inplace = True)
            drop.append(tickers[i])       
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1

            NominalPrice  NetChange  Change    Bid    Ask  PEx   High    Low  \
Date                                                                           
2020-02-17         13.28        NaN     NaN  13.26  13.28  NaN  13.34  13.00   
2020-02-14         13.10        NaN     NaN  13.08  13.10  NaN  13.58  13.00   
2020-02-13         13.38        NaN     NaN  13.38  13.40  NaN  13.64  12.78   

            PreviousClose  ShareVolume000  Turnover000  LotSize  
Date                                                             
2020-02-17          13.10        151583.0    2008730.0      NaN  
2020-02-14          13.38        213336.0    2820528.0      NaN  
2020-02-13          12.92        370025.0    4882186.0      NaN  
            NominalPrice  NetChange  Change   Bid   Ask  PEx  High   Low  \
Date                                                                       
2020-02-17          3.35        NaN     NaN  3.34  3.35  NaN  3.38  3.08   
2020-02-14          3.08        NaN     N

In [18]:
# =============================================================================
# Backtesting strategy - II : Intraday resistance breakout strategy
# Author : 

# Please report bug/issues in the Q&A section
# =============================================================================

import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import copy


def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2['ATR']

def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    n = len(df)/(252*78)
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    vol = df["ret"].std() * np.sqrt(252*78)
    return vol

def sharpe(DF,rf):
    "function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr
    

def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

# Download historical data (monthly) for selected stocks

tickers = ["0700.HKG","1211.HKG","1810.HKG","9988.HKG"]


ohlc_intraday = {} # directory with ohlc value for each stock            
key_path = "/Users/manwaiwong/key/key.txt"
ts = TimeSeries(key=open(key_path,'r').read(), output_format='pandas')

attempt = 0 # initializing passthrough variable
drop = [] # initializing list to store tickers whose close price was successfully extracted
while len(tickers) != 0 and attempt <=5:
    tickers = [j for j in tickers if j not in drop]
    for i in range(len(tickers)):
        try:
            ohlc_intraday[tickers[i]] = ts.get_intraday(symbol=tickers[i],interval='5min', outputsize='full')[0]
            ohlc_intraday[tickers[i]].columns = ["Open","High","Low","Adj Close","Volume"]
            drop.append(tickers[i])      
        except:
            print(tickers[i]," :failed to fetch data...retrying")
            continue
    attempt+=1

tickers = ohlc_intraday.keys() # redefine tickers variable after removing any tickers with corrupted data

################################Backtesting####################################

# calculating ATR and rolling max price for each stock and consolidating this info by stock in a separate dataframe
ohlc_dict = copy.deepcopy(ohlc_intraday)
tickers_signal = {}
tickers_ret = {}
for ticker in tickers:
    print("calculating ATR and rolling max price for ",ticker)
    ohlc_dict[ticker]["ATR"] = ATR(ohlc_dict[ticker],20)
    ohlc_dict[ticker]["roll_max_cp"] = ohlc_dict[ticker]["High"].rolling(20).max()
    ohlc_dict[ticker]["roll_min_cp"] = ohlc_dict[ticker]["Low"].rolling(20).min()
    ohlc_dict[ticker]["roll_max_vol"] = ohlc_dict[ticker]["Volume"].rolling(20).max()
    ohlc_dict[ticker].dropna(inplace=True)
    tickers_signal[ticker] = ""
    tickers_ret[ticker] = []


# identifying signals and calculating daily return (stop loss factored in)
for ticker in tickers:
    print("calculating returns for ",ticker)
    for i in range(len(ohlc_dict[ticker])):
        if tickers_signal[ticker] == "":
            tickers_ret[ticker].append(0)
            if ohlc_dict[ticker]["High"][i]>=ohlc_dict[ticker]["roll_max_cp"][i] and \
               ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker] = "Buy"
            elif ohlc_dict[ticker]["Low"][i]<=ohlc_dict[ticker]["roll_min_cp"][i] and \
               ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker] = "Sell"
        
        elif tickers_signal[ticker] == "Buy":
            if ohlc_dict[ticker]["Adj Close"][i]<ohlc_dict[ticker]["Adj Close"][i-1] - ohlc_dict[ticker]["ATR"][i-1]:
                tickers_signal[ticker] = ""
                tickers_ret[ticker].append(((ohlc_dict[ticker]["Adj Close"][i-1] - ohlc_dict[ticker]["ATR"][i-1])/ohlc_dict[ticker]["Adj Close"][i-1])-1)
            elif ohlc_dict[ticker]["Low"][i]<=ohlc_dict[ticker]["roll_min_cp"][i] and \
               ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker] = "Sell"
                tickers_ret[ticker].append(((ohlc_dict[ticker]["Adj Close"][i-1] - ohlc_dict[ticker]["ATR"][i-1])/ohlc_dict[ticker]["Adj Close"][i-1])-1)
            else:
                tickers_ret[ticker].append((ohlc_dict[ticker]["Adj Close"][i]/ohlc_dict[ticker]["Adj Close"][i-1])-1)
                
        elif tickers_signal[ticker] == "Sell":
            if ohlc_dict[ticker]["Adj Close"][i]>ohlc_dict[ticker]["Adj Close"][i-1] + ohlc_dict[ticker]["ATR"][i-1]:
                tickers_signal[ticker] = ""
                tickers_ret[ticker].append((ohlc_dict[ticker]["Adj Close"][i-1]/(ohlc_dict[ticker]["Adj Close"][i-1] + ohlc_dict[ticker]["ATR"][i-1]))-1)
            elif ohlc_dict[ticker]["High"][i]>=ohlc_dict[ticker]["roll_max_cp"][i] and \
               ohlc_dict[ticker]["Volume"][i]>1.5*ohlc_dict[ticker]["roll_max_vol"][i-1]:
                tickers_signal[ticker] = "Buy"
                tickers_ret[ticker].append((ohlc_dict[ticker]["Adj Close"][i-1]/(ohlc_dict[ticker]["Adj Close"][i-1] + ohlc_dict[ticker]["ATR"][i-1]))-1)
            else:
                tickers_ret[ticker].append((ohlc_dict[ticker]["Adj Close"][i-1]/ohlc_dict[ticker]["Adj Close"][i])-1)
                
    ohlc_dict[ticker]["ret"] = np.array(tickers_ret[ticker])


# calculating overall strategy's KPIs
strategy_df = pd.DataFrame()
for ticker in tickers:
    strategy_df[ticker] = ohlc_dict[ticker]["ret"]
strategy_df["ret"] = strategy_df.mean(axis=1)
CAGR(strategy_df)
sharpe(strategy_df,0.025)
max_dd(strategy_df)  


# vizualization of strategy return
(1+strategy_df["ret"]).cumprod().plot()


#calculating individual stock's KPIs
cagr = {}
sharpe_ratios = {}
max_drawdown = {}
for ticker in tickers:
    print("calculating KPIs for ",ticker)      
    cagr[ticker] =  CAGR(ohlc_dict[ticker])
    sharpe_ratios[ticker] =  sharpe(ohlc_dict[ticker],0.025)
    max_drawdown[ticker] =  max_dd(ohlc_dict[ticker])

KPI_df = pd.DataFrame([cagr,sharpe_ratios,max_drawdown],index=["Return","Sharpe Ratio","Max Drawdown"])      
KPI_df.T



calculating ATR and rolling max price for  1211.HKG
calculating ATR and rolling max price for  0700.HKG
calculating ATR and rolling max price for  9988.HKG
calculating ATR and rolling max price for  1810.HKG
calculating returns for  1211.HKG
calculating returns for  0700.HKG
calculating returns for  9988.HKG
calculating returns for  1810.HKG


/Users/manwaiwong/anaconda3/envs/ai_trade/lib/python3.5/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in double_scalars


ImportError: No module named 'matplotlib'